# Binary Monte Carlo: MEV Contagion + Infrastructure Scenarios

## Purpose
Stochastic simulation testing:
- **Baseline**: Current Jito (80.1% cascade rate)
- **BAM Privacy**: 65% visibility reduction
- **Harmony Multi-Builder**: 40% reduction + competition

Key outputs:
- Mean cascades per scenario
- P90 slots jumped (proxy for skipped-slot risk)
- Economic impact (loss distribution)
- Infrastructure efficiency gap

## Data Sources
- `contagion_report.json`: Real cascade rates, attack probabilities
- `04_validator_analysis/`: Validator stake weights, participation
- Historical oracle lag distributions: BisonFi baseline 180ms

## 1. Import Required Libraries and Load Data

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import json
from pathlib import Path
from datetime import datetime
from typing import Dict, List, Tuple, Optional
import warnings
warnings.filterwarnings('ignore')

# Set style
sns.set_style("whitegrid")
plt.rcParams['figure.figsize'] = (14, 8)
np.random.seed(42)

print("✓ Libraries imported successfully")

✓ Libraries imported successfully


In [2]:
# Import the Monte Carlo module
import sys
sys.path.insert(0, './')
from mev_contagion_monte_carlo import ContagionMonteCarlo

print("✓ Monte Carlo module loaded")

✓ Monte Carlo module loaded


In [3]:
# Load contagion_report.json to extract real data
contagion_report_path = '../contagion_report.json'

try:
    with open(contagion_report_path, 'r') as f:
        contagion_report = json.load(f)
    
    # Extract key metrics
    cascade_analysis = contagion_report.get('sections', {}).get('cascade_rate_analysis', {})
    cascade_rates = cascade_analysis.get('cascade_rates', {})
    
    actual_cascade_pct = cascade_rates.get('cascade_percentage', 0.0)
    trigger_attacks_total = cascade_rates.get('trigger_attacks_total', 593)
    
    print("✓ Contagion Report Loaded")
    print(f"  - Trigger attacks total: {trigger_attacks_total}")
    print(f"  - Historical cascade rate: {actual_cascade_pct:.2f}%")
    print(f"  - Time window: {cascade_rates.get('time_window_ms')}ms")
    
    # Extract downstream pool attack probabilities
    attack_probs = contagion_report.get('sections', {}).get('attack_probability_analysis', {})
    downstream_probs = attack_probs.get('downstream_attack_probabilities', [])
    
    print(f"\n  Downstream pools analyzed: {len(downstream_probs)}")
    for pool in downstream_probs[:3]:  # Show first 3
        print(f"    - {pool['downstream_pool']}: {pool['attack_probability_pct']:.2f}% risk")
    
except FileNotFoundError:
    print(f"⚠ Contagion report not found at {contagion_report_path}")
    print("  Using default parameters instead")

✓ Contagion Report Loaded
  - Trigger attacks total: 593
  - Historical cascade rate: 0.00%
  - Time window: 5000ms

  Downstream pools analyzed: 7
    - BisonFi: 22.43% risk
    - SolFiV2: 21.75% risk
    - GoonFi: 21.59% risk


## 2. Define Monte Carlo Simulation Function

In [7]:
# The ContagionMonteCarlo class implements the full simulation
# Key parameters:
# - base_trigger_prob: 0.15 (15% chance of MEV attack per slot)
# - cascade_rate: 0.801 (80.1% of attacks cascade to downstream pools)
# - visibility_reduction: Infrastructure-specific modifier
#   - Jito: 0.0 (no reduction, baseline)
#   - BAM: 0.65 (65% visibility hidden)
#   - Harmony: 0.40 (40% reduction + competition effects)

print("""
Monte Carlo Simulation Logic:
=============================

For each infrastructure scenario (100,000 simulations):

1. Binary Trigger: Roll dice - does an attack trigger this slot?
   Trigger = Bernoulli(base_trigger_prob=0.15)

2. Binary Cascades: If triggered, how many cascade attempts?
   effective_cascade_rate = cascade_rate × (1 - visibility_reduction)
   cascades = Binomial(runs_per_slot=5, p=effective_cascade_rate)

3. Slot Jumps: Each cascade consumes time (100-700ms)
   total_time = sum(cascade_times)
   slots_jumped = ceil(total_time / 400ms)

4. Economic Loss: Base loss + oracle lag impact
   loss = cascades × (50 + oracle_lag_ms × 0.3)

5. Infrastructure Gap: How much this setup protected vs baseline
   gap = (baseline_loss - scenario_loss) / baseline_loss
""")


Monte Carlo Simulation Logic:

For each infrastructure scenario (100,000 simulations):

1. Binary Trigger: Roll dice - does an attack trigger this slot?
   Trigger = Bernoulli(base_trigger_prob=0.15)

2. Binary Cascades: If triggered, how many cascade attempts?
   effective_cascade_rate = cascade_rate × (1 - visibility_reduction)
   cascades = Binomial(runs_per_slot=5, p=effective_cascade_rate)

3. Slot Jumps: Each cascade consumes time (100-700ms)
   total_time = sum(cascade_times)
   slots_jumped = ceil(total_time / 400ms)

4. Economic Loss: Base loss + oracle lag impact
   loss = cascades × (50 + oracle_lag_ms × 0.3)

5. Infrastructure Gap: How much this setup protected vs baseline
   gap = (baseline_loss - scenario_loss) / baseline_loss



## 3. Configure Baseline and Alternative Scenarios

In [9]:
# Initialize Monte Carlo simulator
mc = ContagionMonteCarlo(output_dir='./outputs')

print("\n" + "="*80)
print("INFRASTRUCTURE SCENARIOS CONFIGURATION")
print("="*80)

for scenario_key, scenario in mc.scenarios.items():
    print(f"\n▶ {scenario['name']}")
    print(f"  Description: {scenario['description']}")
    print(f"  Base trigger prob: {scenario['base_trigger_prob']:.2%}")
    print(f"  Cascade rate: {scenario['cascade_rate']:.1%}")
    print(f"  Visibility reduction: {scenario['visibility_reduction']:.1%}")
    
    if 'competition_factor' in scenario:
        print(f"  Competition factor: {scenario['competition_factor']:.1%}")

print(f"\n\nNetwork Parameters:")
print(f"  Oracle lag (baseline): {mc.network_params['oracle_lag_ms']}ms")
print(f"  Solana slot time: {mc.network_params['slot_time_ms']}ms")
print(f"  Cascade attempts per slot: {mc.network_params['runs_per_slot']}")
print(f"  Skipped-slot threshold: {mc.network_params['skipped_slot_threshold']} slots")


INFRASTRUCTURE SCENARIOS CONFIGURATION

▶ Jito Baseline (Current)
  Description: Current state with centralized MEV auctions
  Base trigger prob: 15.00%
  Cascade rate: 80.1%
  Visibility reduction: 0.0%

▶ BAM Privacy (65% visibility reduction)
  Description: With encrypted transactions & threshold encryption
  Base trigger prob: 15.00%
  Cascade rate: 80.1%
  Visibility reduction: 65.0%

▶ Harmony Multi-Builder (40% reduction + competition)
  Description: With multi-builder competition & validator separation
  Base trigger prob: 15.00%
  Cascade rate: 80.1%
  Visibility reduction: 40.0%
  Competition factor: 80.0%


Network Parameters:
  Oracle lag (baseline): 180ms
  Solana slot time: 400ms
  Cascade attempts per slot: 5
  Skipped-slot threshold: 3 slots


## 4. Run Stochastic Simulations

In [10]:
# Run all scenarios (100k iterations each)
# This may take 10-30 seconds depending on machine

print("\n" + "="*80)
print("RUNNING MONTE CARLO SIMULATIONS")
print("="*80)

import time
start_time = time.time()

results = mc.run_all_scenarios(
    n_sims=100_000,
    contagion_report='../contagion_report.json'
)

elapsed = time.time() - start_time
print(f"\n✓ Simulations completed in {elapsed:.2f} seconds")
print(f"  Total iterations: {len(mc.scenarios) * 100_000:,}")


RUNNING MONTE CARLO SIMULATIONS

BINARY MONTE CARLO: MEV CONTAGION + INFRASTRUCTURE SCENARIOS
✓ Loaded contagion report
  Actual cascade rate: 0.00%

▶ Running Jito Baseline (Current)...
  Simulations: 100,000
  ✓ Attack rate: 14.90%
  ✓ Mean cascades (when triggered): 3.99
  ✓ P90 slots jumped: 6.00
  ✓ High risk events: 11.62%
  ✓ Mean infra gap: 0.00%

▶ Running BAM Privacy (65% visibility reduction)...
  Simulations: 100,000
  ✓ Attack rate: 14.97%
  ✓ Mean cascades (when triggered): 1.41
  ✓ P90 slots jumped: 3.00
  ✓ High risk events: 1.45%
  ✓ Mean infra gap: 10.71%

▶ Running Harmony Multi-Builder (40% reduction + competition)...
  Simulations: 100,000
  ✓ Attack rate: 15.03%
  ✓ Mean cascades (when triggered): 1.93
  ✓ P90 slots jumped: 4.00
  ✓ High risk events: 2.90%
  ✓ Mean infra gap: 9.22%

✓ Simulations completed in 0.51 seconds
  Total iterations: 300,000


## 5. Analyze and Compare Results

In [11]:
# Print comprehensive comparison table
print("\n" + "="*80)
print("INFRASTRUCTURE COMPARISON TABLE")
print("="*80)

comparison_table = mc.generate_comparison_table()
print("\n" + comparison_table.to_string(index=False))


INFRASTRUCTURE COMPARISON TABLE

                                     Infrastructure Attack Rate % Mean Cascades P90 Cascades P90 Slots Mean Loss High Risk % Inefficiency vs Baseline
                            Jito Baseline (Current)        14.90%          3.99         5.00      6.00   $415.23      11.62%                     0.0%
             BAM Privacy (65% visibility reduction)        14.97%          1.41         3.00      3.00   $148.22       1.45%                    10.7%
Harmony Multi-Builder (40% reduction + competition)        15.03%          1.93         3.00      4.00   $201.01       2.90%                     9.2%


In [12]:
# Detailed statistics per scenario
print("\n" + "="*80)
print("DETAILED STATISTICS BY SCENARIO")
print("="*80)

for scenario_key in mc.results.keys():
    df = mc.results[scenario_key]
    stats = mc.summary_stats[scenario_key]
    scenario = mc.scenarios[scenario_key]
    
    print(f"\n{'='*80}")
    print(f"{scenario['name']}")
    print(f"{'='*80}")
    
    # Triggered attacks only
    triggered = df[df['trigger'] == 1]
    
    print(f"\nAttack Statistics:")
    print(f"  Total simulations: {len(df):,}")
    print(f"  Attack rate: {stats['attack_rate_pct']:.2f}%")
    print(f"  Triggered attacks: {len(triggered):,}")
    
    print(f"\nCascade Statistics (when triggered):")
    print(f"  Mean cascades: {stats['mean_cascades']:.3f}")
    print(f"  Median cascades: {stats['median_cascades']:.3f}")
    print(f"  P90 cascades: {stats['p90_cascades']:.3f}")
    print(f"  P99 cascades: {stats['p99_cascades']:.3f}")
    
    print(f"\nSlot Jump Statistics (congestion proxy):")
    print(f"  Mean slots jumped: {stats['mean_slots_jumped']:.3f}")
    print(f"  P90 slots jumped: {stats['p90_slots_jumped']:.3f}")
    print(f"  P99 slots jumped: {stats['p99_slots_jumped']:.3f}")
    
    print(f"\nEconomic Impact:")
    print(f"  Mean loss: ${stats['mean_loss']:.2f}")
    print(f"  P90 loss: ${stats['p90_loss']:.2f}")
    
    print(f"\nRisk Metrics:")
    print(f"  High-risk events (slots > {mc.network_params['skipped_slot_threshold']}): {stats['high_risk_pct']:.2f}%")
    print(f"  Infrastructure gap vs baseline: {stats['mean_infra_gap']:.1%}")


DETAILED STATISTICS BY SCENARIO

Jito Baseline (Current)

Attack Statistics:
  Total simulations: 100,000
  Attack rate: 14.90%
  Triggered attacks: 14,903

Cascade Statistics (when triggered):
  Mean cascades: 3.994
  Median cascades: 4.000
  P90 cascades: 5.000
  P99 cascades: 5.000

Slot Jump Statistics (congestion proxy):
  Mean slots jumped: 4.486
  P90 slots jumped: 6.000
  P99 slots jumped: 7.000

Economic Impact:
  Mean loss: $415.23
  P90 loss: $530.01

Risk Metrics:
  High-risk events (slots > 3): 11.62%
  Infrastructure gap vs baseline: 0.0%

BAM Privacy (65% visibility reduction)

Attack Statistics:
  Total simulations: 100,000
  Attack rate: 14.97%
  Triggered attacks: 14,973

Cascade Statistics (when triggered):
  Mean cascades: 1.410
  Median cascades: 1.000
  P90 cascades: 3.000
  P99 cascades: 4.000

Slot Jump Statistics (congestion proxy):
  Mean slots jumped: 1.818
  P90 slots jumped: 3.000
  P99 slots jumped: 5.000

Economic Impact:
  Mean loss: $148.22
  P90 loss:

In [13]:
# Relative risk reduction
print("\n" + "="*80)
print("INFRASTRUCTURE PROTECTION ANALYSIS")
print("="*80)

baseline_stats = mc.summary_stats['jito_baseline']

for scenario_key in ['bam_privacy', 'harmony_multibuilder']:
    scenario = mc.scenarios[scenario_key]
    stats = mc.summary_stats[scenario_key]
    
    cascade_reduction = 1 - (stats['mean_cascades'] / baseline_stats['mean_cascades']) if baseline_stats['mean_cascades'] > 0 else 0
    slots_reduction = 1 - (stats['p90_slots_jumped'] / baseline_stats['p90_slots_jumped']) if baseline_stats['p90_slots_jumped'] > 0 else 0
    loss_reduction = 1 - (stats['mean_loss'] / baseline_stats['mean_loss']) if baseline_stats['mean_loss'] > 0 else 0
    risk_reduction = 1 - (stats['high_risk_pct'] / baseline_stats['high_risk_pct'] * 100) if baseline_stats['high_risk_pct'] > 0 else 0
    
    print(f"\n{scenario['name']} vs Jito Baseline:")
    print(f"  Cascade reduction: {cascade_reduction:.1%}")
    print(f"  P90 slots reduction: {slots_reduction:.1%}")
    print(f"  Economic loss reduction: {loss_reduction:.1%}")
    print(f"  High-risk event reduction: {risk_reduction:.1%}")


INFRASTRUCTURE PROTECTION ANALYSIS

BAM Privacy (65% visibility reduction) vs Jito Baseline:
  Cascade reduction: 64.7%
  P90 slots reduction: 50.0%
  Economic loss reduction: 64.3%
  High-risk event reduction: -1149.2%

Harmony Multi-Builder (40% reduction + competition) vs Jito Baseline:
  Cascade reduction: 51.8%
  P90 slots reduction: 33.3%
  Economic loss reduction: 51.6%
  High-risk event reduction: -2396.8%


## 6. Visualize Distributions and Risk Metrics

In [ ]:
# Generate cascade distribution visualization
mc.plot_cascade_distributions(figsize=(16, 10))
plt.show()

In [ ]:
# Generate infrastructure comparison visualization
mc.plot_infrastructure_comparison(figsize=(16, 12))
plt.show()

In [ ]:
# Create detailed box plots
fig, axes = plt.subplots(2, 2, figsize=(14, 10))

# Prepare data for plotting
all_data = []
for scenario_key, df in mc.results.items():
    triggered = df[df['trigger'] == 1]
    for col, col_name in [('cascades', 'Cascades'), ('slots_jumped', 'Slots'), ('total_loss', 'Loss ($)')]:
        for val in triggered[col]:
            all_data.append({
                'Scenario': mc.scenarios[scenario_key]['name'].split('(')[0],
                'Metric': col_name,
                'Value': val
            })

plot_df = pd.DataFrame(all_data)

# Box plots for each metric
for i, metric in enumerate(['Cascades', 'Slots', 'Loss ($)']):
    ax = axes.flat[i]
    data = plot_df[plot_df['Metric'] == metric]
    sns.boxplot(x='Scenario', y='Value', data=data, ax=ax, palette='Set2')
    ax.set_title(f'{metric} Distribution by Infrastructure', fontweight='bold')
    ax.set_xlabel('')
    if i < 2:
        ax.get_xaxis().set_visible(False)

# Risk comparison
ax = axes.flat[3]
scenarios_list = list(mc.scenarios.keys())
high_risk_pcts = [mc.summary_stats[s]['high_risk_pct'] for s in scenarios_list]
colors = sns.color_palette('Set2', len(scenarios_list))
bars = ax.bar(range(len(scenarios_list)), high_risk_pcts, color=colors)
ax.set_ylabel('High Risk (%)')
ax.set_title('High-Risk Event Rate by Infrastructure', fontweight='bold')
ax.set_xticks(range(len(scenarios_list)))
ax.set_xticklabels([mc.scenarios[s]['name'].split('(')[0] for s in scenarios_list], rotation=15, ha='right')
for bar in bars:
    height = bar.get_height()
    ax.text(bar.get_x() + bar.get_width()/2., height, f'{height:.1f}%', ha='center', va='bottom')

plt.tight_layout()
filename = f'monte_carlo_boxplots_{datetime.now().strftime("%Y%m%d_%H%M%S")}.png'
filepath = mc.output_dir / filename
plt.savefig(filepath, dpi=150, bbox_inches='tight')
print(f"✓ Saved visualization → {filename}")
plt.show()

In [ ]:
# Oracle lag vs slots jumped correlation
fig, axes = plt.subplots(1, 3, figsize=(16, 4))

for i, (scenario_key, ax) in enumerate(zip(mc.results.keys(), axes)):
    df = mc.results[scenario_key]
    triggered = df[df['trigger'] == 1]
    
    # Scatter with density coloring
    h = ax.hist2d(triggered['oracle_lag_ms'], triggered['slots_jumped'], 
                   bins=30, cmap='YlOrRd')
    ax.set_xlabel('Oracle Lag (ms)')
    ax.set_ylabel('Slots Jumped')
    ax.set_title(f"{mc.scenarios[scenario_key]['name'].split('(')[0]}", fontweight='bold')
    plt.colorbar(h[3], ax=ax, label='Frequency')

plt.tight_layout()
filename = f'oracle_lag_correlation_{datetime.now().strftime("%Y%m%d_%H%M%S")}.png'
filepath = mc.output_dir / filename
plt.savefig(filepath, dpi=150, bbox_inches='tight')
print(f"✓ Saved visualization → {filename}")
plt.show()

## 7. Validate Against Historical Contagion Data

In [ ]:
# Compare Monte Carlo output with historical contagion data
print("\n" + "="*80)
print("MODEL VALIDATION: MONTE CARLO vs HISTORICAL DATA")
print("="*80)

# Extract historical cascade rate
historical_cascade_pct = cascade_rates.get('cascade_percentage', 0.0)

# Simulated baseline cascade rate
baseline_df = mc.results['jito_baseline']
baseline_cascade_rate = baseline_df['cascades'].mean() / mc.network_params['runs_per_slot'] * 100
simulated_attack_rate = baseline_df['trigger'].mean() * 100

print(f"\nCascade Rate Comparison:")
print(f"  Historical cascade %: {historical_cascade_pct:.2f}%")
print(f"  Simulated cascade % (effective): {baseline_cascade_rate:.2f}%")
print(f"  Simulated attack trigger rate: {simulated_attack_rate:.2f}%")

if abs(historical_cascade_pct - baseline_cascade_rate) < 10:
    print(f"  ✓ Good fit! Difference within ±10%")
else:
    print(f"  ⚠ Model calibration needed. Difference: {abs(historical_cascade_pct - baseline_cascade_rate):.1f}%")

# Goodness of fit
print(f"\nRisk Metrics Validation:")
print(f"  P90 slots jumped (baseline): {mc.summary_stats['jito_baseline']['p90_slots_jumped']:.2f}")
print(f"  High-risk event rate: {mc.summary_stats['jito_baseline']['high_risk_pct']:.2f}%")

# Downstream pool attack probabilities
if downstream_probs:
    print(f"\nDownstream Pool Risk Analysis:")
    avg_downstream_prob = np.mean([p['attack_probability_pct'] for p in downstream_probs])
    print(f"  Average downstream attack probability: {avg_downstream_prob:.2f}%")
    print(f"  Range: {min(p['attack_probability_pct'] for p in downstream_probs):.2f}% - {max(p['attack_probability_pct'] for p in downstream_probs):.2f}%")

In [ ]:
# Save all results to CSV
print("\n" + "="*80)
print("SAVING RESULTS")
print("="*80)

mc.save_results(tag='binary_monte_carlo')

print(f"\n✓ All results saved to {mc.output_dir}")

## Summary and Key Findings

In [ ]:
# Final summary
print("\n" + "="*80)
print("FINAL SUMMARY: MEV CONTAGION Monte CARLO ANALYSIS")
print("="*80)

baseline_stats = mc.summary_stats['jito_baseline']
bam_stats = mc.summary_stats['bam_privacy']
harmony_stats = mc.summary_stats['harmony_multibuilder']

print(f"""
KEY FINDINGS:
=============

1. JITO BASELINE (Current State)
   - Attack rate: {baseline_stats['attack_rate_pct']:.2f}%
   - Average cascades: {baseline_stats['mean_cascades']:.2f}
   - P90 slots jumped: {baseline_stats['p90_slots_jumped']:.2f}
   - Economic loss: ${baseline_stats['mean_loss']:.2f}
   - High-risk events: {baseline_stats['high_risk_pct']:.2f}%

2. BAM PRIVACY (+65% visibility reduction)
   - Cascade reduction: {(1 - bam_stats['mean_cascades']/baseline_stats['mean_cascades'])*100:.1f}%
   - Slots reduction: {(1 - bam_stats['p90_slots_jumped']/baseline_stats['p90_slots_jumped'])*100:.1f}%
   - Loss reduction: {(1 - bam_stats['mean_loss']/baseline_stats['mean_loss'])*100:.1f}%
   - High-risk reduction: {(1 - harmony_stats['high_risk_pct']/baseline_stats['high_risk_pct'])*100:.1f}%

3. HARMONY MULTI-BUILDER (+40% reduction + competition)
   - Cascade reduction: {(1 - harmony_stats['mean_cascades']/baseline_stats['mean_cascades'])*100:.1f}%
   - Slots reduction: {(1 - harmony_stats['p90_slots_jumped']/baseline_stats['p90_slots_jumped'])*100:.1f}%
   - Loss reduction: {(1 - harmony_stats['mean_loss']/baseline_stats['mean_loss'])*100:.1f}%
   - Competition benefit: Additional cascade suppression

INFRASTRUCTURE RECOMMENDATION:
=============================
Both BAM and Harmony significantly reduce MEV contagion risk.
- BAM provides strongest immediate protection (privacy)
- Harmony offers balanced approach (multi-builder competition)
- Combined strategy (BAM + competition) maximally effective
""")